数据库结构  
数据库名：stock_data  
表名：data_status
|字段|类型|值|
|:-:|:-:|:-:|
|stock_name|VARCHAR(20)|股票名|
|last_date|VARCHAR(10)|数据尾部的日期|
|last_update|VARCHAR(10)|上次更新的时间|

In [1]:
import pymysql
import time
import yfinance as yf
from pandas_datareader import data
import datetime
import os
import pandas as pd
import torch
from kits import *
from Tool_v1 import *
yf.pdr_override()
db_map = {"test": "stock_data_test", "formal": "stock_data"}

In [14]:
def fetch_stock_data(stock_name, start="default", end = "default"):
    if end == "default":
        now_date, now_time = get_now_date()
        end = now_date
    if start == "default":
        start = get_previous_date(end)
    stock_data = data.get_data_yahoo(tickers=stock_name,start=start,end=end,progress=False)
    if len(stock_data) == 0:
        return None, None
    return stock_data, str(stock_data.index.tolist()[-1]).split(' ')[0]


def read_db(stock_name, date, env):
    update(stock_name, env)
    now_date = get_now_date()[0]
    if date > now_date:
        return None
    else:
        previous = get_previous_date(now_date)
        database = db_map[env]
        db = pymysql.connect(
            user = "root",
            passwd = "123456",
            database = database
        )
        cursor = db.cursor()
        cursor.execute("SELECT * FROM %s WHERE Date < '%s' and Date >= '%s'"%(stock_name, date, previous))
        d = cursor.fetchall()
        df = pd.core.frame.DataFrame(d)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
        df.set_index("Date", inplace=True)
        return df

In [15]:
df = read_db("aapl", "2023-11-14", "test")

              0       1       2       3       4        5          6
0    2021-11-18  153.71  158.67  153.05  157.87  156.087  137827700
1    2021-11-19  157.65  161.02  156.53  160.55  158.736  117305600
2    2021-11-22  161.68  165.70  161.00  161.02  159.201  117467900
3    2021-11-23  161.12  161.80  159.06  161.41  159.587   96041900
4    2021-11-24  160.75  162.14  159.64  161.94  160.111   69463600
..          ...     ...     ...     ...     ...      ...        ...
494  2023-11-07  179.18  182.44  178.97  181.82  181.581   70530000
495  2023-11-08  182.35  183.45  181.59  182.89  182.649   49340300
496  2023-11-09  182.96  184.12  181.81  182.41  182.170   53763500
497  2023-11-10  183.97  186.57  183.53  186.40  186.400   66133400
498  2023-11-13  185.82  186.03  184.21  184.80  184.800   43627500

[499 rows x 7 columns]


In [17]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-11-18,153.71,158.67,153.05,157.87,156.087,137827700
2021-11-19,157.65,161.02,156.53,160.55,158.736,117305600
2021-11-22,161.68,165.70,161.00,161.02,159.201,117467900
2021-11-23,161.12,161.80,159.06,161.41,159.587,96041900
2021-11-24,160.75,162.14,159.64,161.94,160.111,69463600
...,...,...,...,...,...,...
2023-11-07,179.18,182.44,178.97,181.82,181.581,70530000
2023-11-08,182.35,183.45,181.59,182.89,182.649,49340300
2023-11-09,182.96,184.12,181.81,182.41,182.170,53763500


In [86]:
stock_data, last_date = fetch_stock_data("aapl")

In [89]:
update("aapl", "test")

2023-11-14
2023-11-14


In [32]:
def get_previous_date(now_time, delta_year=2):
    [now_year, now_mon, now_day] = now_time.split("-")
    return "-".join([str(int(now_year) - delta_year), now_mon, now_day])

In [33]:
get_previous_date(get_now_date())

'2021-11-13'